In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import os
import csv
import xgboost
import gensim
import matplotlib.pyplot as plt
import scikitplot as skplot

from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn import utils
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalise
from sklearn.linear_model import SGDClassifier, LogisticRegression


from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument



In [ ]:
STOP_WORDS = 'english'
#NROWS_IMPORT = 300000

In [ ]:
df = pd.read_csv('data.csv', n_rows=NROWS_IMPORT)
df.dropna(inplace=True, subset=['EMAIL_TEXT,'INITIAL_EMAIL_SUBJECT_LINE''])



In [ ]:
df.shape

In [ ]:
string_count = Counter(df[EMAIL_TEXT])
common = pd.DataFrame(string_count.most_common(10000))

print(common)

In [ ]:
#.str.count can be used to cound a number of text strings in a data frame
#df['EMAIL_TEXT]'.str.count('fill this with a string or whatever')

In [ ]:
def text_preprocessing(df):
    
    df['EMAIL_TEXT'] = df['EMAIL_TEXT'].apply(lambda x : str(x))
    df['EMAIL_TEXT'] = df['EMAIL_TEXT'].apply(lambda x : x.lower())
    df['EMAIL_TEXT'] = df['EMAIL_TEXT'].str.replace('\n','').str.replace('\r','')
    df['EMAIL_TEXT_CLEAN'] = df['EMAIL_TEXT'].apply(lambda x : x.replace("literally any string you can put in here and copy it over and over",""))
                                                                .replace("fill this in again with literally anything","")
        
    df['EMAIL_TEXT_CLEAN'] = df['EMAIL_TEXT_CLEAN'].apply(lambda text: gensim.utils.simple_preprocess(text))
    df['EMAIL_TEXT_CLEAN'] = df['EMAIL_TEXT_CLEAN'].apply(lambda text: ' '.join(text))
    

In [ ]:
text_preprocessing(df)

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(
df.EMAIL_TEXT_CLEAN, df.ACTION_TAKEN_TARGET, test_size=.2, random_state=69)


In [ ]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS)),
#     'sgd',SGDClassifier())
    ('xgbc',xgboost.XGBClassifier(nthread=-1, n_job=4, class_weight='balanced'))
])

In [ ]:
pred = cross_val_predict(pipe, df.EMAIL_TEXT_CLEAN, df.ACTION_TAKEN_TARGET, cv=4)

In [ ]:
pipe.fit(X_train, y_train)
probs = pipe.predict_proba(X_test)

In [ ]:
print(classification_report(y_test, np.argmax(probs, axis=1)))
#rinse and repeat

In [ ]:
skplt.metrics.plot_roc(y_test, probs)

# what some others did

In [ ]:
import pickle
import warnings
warnings.filterwarnings("ignore")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
df.EMAIL_TEXT_CLEAN, df.ACTION_TAKEN_TARGET, test_size=.2,
    random_state=69, stratify=y_train['ACTION_TAKEN_TARGET'])


In [ ]:
# so instead of changing it before try changing tfidf after
df2 = tdif.fit_transform(df['EMAIL_TEXT_CLEAN'])

In [ ]:
# logistic regression = model.fit(X_train, y_train)

In [ ]:
# # weighted logreg
# # W.L.R = LogisticRegression(C=1.0, classweight='balanced, dual=False, fit_intercept=True,
#     intercept_scaling=1, max_iter=100, multi_class='multinomial',
#     n_jobs=-1, penalty='l2', random_state=69, solver='lbfgs', 
#     tol=0.0001, verbose=0, warm_start=False)

In [ ]:
# W.L.R.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)

In [ ]:
print("CV Scores:\n{}".format(
    cross_val_score(W.L.R, X_train, y_train, cv=kfold)))